## Requirements

### Web-Scraping
#### 1. Take the base wg-gesucht URL and scrape the shared flats listed on the website
#### 2. most important data points are price, room size and number of flat mates

### Data Analyis
#### 1. Correlation analysis of room price and room size. Hypothesis: positive correlation
#### 2. Correlation analysis of room price and number of flat mates. Hypothesis: negative correlation

In [12]:
import requests

In [15]:
BASE_URL = 'https://www.wg-gesucht.de/wg-zimmer-in-Mannheim.85.0.0.0.html'

res = requests.get(BASE_URL)

In [16]:
print(res.text)


<!DOCTYPE html>
<html>
<head>
        <meta charset="utf-8"/>
    <meta http-equiv="X-UA-Compatible" content="IE=edge"/>
    <meta name="viewport" content="width=device-width" id="viewport"/>
        <script data-cmp-ab="1">
        window.cmp_block_ignoredomains = [
            "stripe.com",
            "meineschufa.de",
            "my.matterport.com",
            "recaptcha.net",
            "paypal.com"
        ];
    </script>
    <script type="text/javascript" data-cmp-ab="1" src="https://cdn.consentmanager.net/delivery/autoblock/15144.js" data-cmp-id="15144" data-cmp-host="a.delivery.consentmanager.net" data-cmp-cdn="cdn.consentmanager.net"></script>
    <meta name="Author" content="WG-Gesucht.de"/>

        <script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
    new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
    j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
    'https://www.googletagmanager.com/gtm.js?id='+i+dl;